In [1]:
# BASE
# ------------------------------------------------------
import pandas as pd  
import numpy as np

In [ ]:
model_train = pd.read_csv('holidays_events.csv')
oil = pd.read_csv('oil.csv')

In [ ]:
stores = model_train.store_nbr.unique()

In [ ]:
stores

In [ ]:
model_dict = {}
for s in stores:
    trainframe = model_train[model_train["store_nbr"]== s].copy()
    model = modelbuilder(trainframe)
    model_dict[s] = model